<h1> <center> <br> SIP2021 x MTL INTERNSHIP <br> <br>  Data Science for Propensity to Buy </center> </h1>

---

<h2> ---------- < Team Member > ---------- </h2>
<h3> 1. Pasakorn Limchuchua <br> <br> 2. Thiranai Keatpimol </h3>

---

<h2> ---------- < Table of Contents > ---------- </h2>

* [1. Introduction](#1.Introduction)
    * [Project Assignment](#ProjectAssignment)
    * [Project Scope](#ProjectScope)    
* [2. Data Preprocessing](#2.DataPreprocessing)
    * [Step 1 : Handle Categorical Variables](#Step1HCV)
    * [Step 2 : Handle Numerical Variables](#Step2HNV)  
    * [Step 3 : Handle High Correlation Variables](#Step3HHCV)
    * [Step 4 : Outlier Handling](#Step4OH)  
* [3. Model Development](#3.ModelDevelopment)
    * [Step 1 : Variable Selection](#Step1VS)
    * [Step 2,3,4 : Model Selection - Model Validation - Performance Testing](#Step234)
* [4. Model Implementation](#4.ModelImplementation)
    * [Interpret with Confusion Matrix](#ICM)
    * [Interpret with SHAP Value](#ISV)

---

<a class='anchor' id='1.Introduction'> <h2> ---------- < 1. Introduction > ---------- </h2> </a> 

<a class='anchor' id='ProjectAssignment'> <h3> Project Assignment </h3> </a>

:  To build model to predict customer propensity to buy product and find customer characteristics

<a class='anchor' id='ProjectScope'> <h3> Project Scope </h3> </a>

: We used the data as of '2019' to predict customers who are likely to repurchase <b> target product </b> in '2020' from an <b> Agent </b> channel.

---

<a class='anchor' id='2.DataPreprocessing'> <h2> ---------- < 2. Data Preprocessing > ---------- </h2> </a>

<b> Import libraries and set display </b>

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import shap
import statsmodels.api as sm
import statistics
%matplotlib inline
from matplotlib.pyplot import figure
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, recall_score, precision_score, f1_score, fbeta_score, classification_report, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier

pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)

<b> Load tabular data </b>

In [ ]:
df = pd.read_csv(r'D:\2021\Internship\cu_internship_data_2019_v2.csv')

In [ ]:
df_clean = df.copy()

<a class='anchor' id='Step1HCV'> <h3> <b> Step 1 : Handle Categorical Variables </b> </h3> </a>

<h3> 1. Dropping unrelated records </h3>

---

<b> <i> GNDR </i> </b>

We decided to drop GNDR value = C because we did not focus on corperate data

---

<h3> 2. Dropping variables </h3>

---

<b> <i> 'BUSI_DT' </i></b>

We decided to drop 'BUSI_DT' because all of them had the same value

In [ ]:
df_clean.drop(columns = ['BUSI_DT'], axis = 1, inplace = True)

Note that we want to keep 'PH_ID_BASE' which has unique value to filter other columns, but we will drop it later

In [ ]:
df_clean = df_clean[df_clean['GNDR'] != 'C']

---

<b> <i> 'JUV_LA_FLG' </i> </b>

We found that 'JUV_LA_FLG' had a large difference between N and Y <br>
We will check that 'JUV_LA_FLG' value = Y has affect on all target = 1 

From result, it had no effect -> We decided to drop 'JUV_LA_FLG' column

In [ ]:
df_clean.drop(columns = ['JUV_LA_FLG'], axis = 1, inplace = True)

---

<h3> 3. Filling missing records </h3>

---

<b> <i> 'MIB_LV' </i> </b>

Note that 'MIB_LV' has 1.0 = critical health, 0 or NaN (from P'Folk) = good health and so on <br>
We decided to fill NaN as 0.0

In [ ]:
df_clean.fillna({'MIB_LV' : 0}, inplace = True)

---

<h3> 4. One-Hot Encoding </h3>

---

In [ ]:
df_clean = pd.get_dummies(df_clean, columns =['EMPL_FLG', 'FIRST_AQUS_CHN', 'GNDR', 'INF_LEG_ANNT_FLG', 'INF_TKF_FLG',
                                   'LAST_CHN', 'LIFE_STG', 'MAT_DUR_CHRCTR', 'ON_ANNT_FLG', 'PC_DUR_CHRCTR', 'PHLA_FLG'])

We dropped out one column per categorical data for preventing redundancy from get dummies<br>
Except 'LIFE_STG' we will drop this column later. (we will use this column to fill NaN value)

In [ ]:
df_clean.drop(columns = ['EMPL_FLG_N', 'FIRST_AQUS_CHN_AF BANK', 'GNDR_F', 'INF_LEG_ANNT_FLG_N', 'INF_TKF_FLG_N', 
                    'LAST_CHN_AF BANK', 'MAT_DUR_CHRCTR_0', 'ON_ANNT_FLG_N', 
                    'PC_DUR_CHRCTR_0', 'PHLA_FLG_N'], axis = 1, inplace = True)

---

<h3> 5. Order Encoding </h3>

---

In [ ]:
ct_dict = { 'SMILECLUB' : 1, 'PLATINUM' : 2, 'PRESTIGE' : 3}
df_clean['CUST_TIER'] = df['CUST_TIER'].map(ct_dict)

We need to fill NaN as 0

In [ ]:
df_clean.fillna({'CUST_TIER' : 0.0}, inplace = True)

In [ ]:
#Replace Unknown Value with 1, Unknown Start with not Digit~Alphabet and End with Digit(or not)
df_clean['CUST_TIER'].replace(regex=r'^[\D]+[\d]*$', value=1)

We need to scaler 'CUST_TIER' using Standardization (STD) ad MINMAX Normalization (MINMAX)

In [ ]:
mmscaler = MinMaxScaler()
sdscaler = StandardScaler()

In [ ]:
df_mm_r = df_clean[['PH_ID_BASE','CUST_TIER']].copy()
df_sd_r = df_clean[['PH_ID_BASE','CUST_TIER']].copy()

In [ ]:
df_mm_r_ft = mmscaler.fit_transform(df_mm_r)
df_mm_r_ft = df_mm_r_ft[:,1].tolist()

In [ ]:
df_clean['MINMAX_CUST_TIER'] = df_mm_r_ft

In [ ]:
df_sd_r_ft = sdscaler.fit_transform(df_sd_r)
df_sd_r_ft = df_sd_r_ft[:,1].tolist()

In [ ]:
df_clean['STD_CUST_TIER'] = df_sd_r_ft

In [ ]:
df_clean.drop(columns = ['CUST_TIER'], axis = 1, inplace = True)

---

<a class='anchor' id='Step2HNV'> <h3> <b> Step 2 : Handle Numerical Variables</b> </h3> </a>

<h3> 1. Filling missing records </h3>

<b> <i> 'LOSS_RATIO_P3Y' </i> </b>

We found that 'STD_LOSS_RATIO_P3Y' majority in value was -0.001542 <br>
We decided to fill 'STD_LOSS_RATIO_P3Y' value = NaN as -0.001542

In [ ]:
df_clean.fillna({'STD_LOSS_RATIO_P3Y' : -0.001542}, inplace = True)

<b> <i> 'LOSS_RATIO_P3Y' </i> </b>

We found that 'MINMAX_LOSS_RATIO_P3Y' majority in value was 0.000000 <br>
We decided to fill 'MINMAX_LOSS_RATIO_P3Y' value = NaN as 0.000000

In [ ]:
df_clean.fillna({'MINMAX_LOSS_RATIO_P3Y' : 0.000000}, inplace = True)

---

<b> <i> 'NUM_CHLDRN' </i> <b>

We found that 'NUM_CHLDRN' has relationship with 'LIFE_STG' So, We use this relationship to fill NaN <br> <br>
Our determination : <br>
1. Observation with 'LIFE_STG' column : Family without children / Single adult / Single young adult / Kid and Teenager <br>
We decided to fill NaN as 0 for Normalization and -1.189127 for Standardization<br>

In [ ]:
df_clean['STD_NUM_CHLDRN'] = np.where((df_clean['STD_NUM_CHLDRN'].isnull()) & (
    (df_clean['LIFE_STG_Family without children'] == 1) | (df_clean['LIFE_STG_Single adult'] == 1) 
    | (df_clean['LIFE_STG_Single young adult'] == 1) | (df_clean['LIFE_STG_Kid and Teenager'] == 1)),
                                         -1.189127, df_clean['STD_NUM_CHLDRN'])

In [ ]:
df_clean['MINMAX_NUM_CHLDRN'] = np.where((df_clean['MINMAX_NUM_CHLDRN'].isnull()) & (
    (df_clean['LIFE_STG_Family without children'] == 1) | (df_clean['LIFE_STG_Single adult'] == 1) 
    | (df_clean['LIFE_STG_Single young adult'] == 1) | (df_clean['LIFE_STG_Kid and Teenager'] == 1)),
                                         0, df_clean['MINMAX_NUM_CHLDRN'])

For 'LIFE_STG' column : New married couple, we found that it's all value was 0 <br>
We decided to fill NaN as 0 for Normalization and -1.189127 for Standardization<br>

In [ ]:
nmc_notnull = len(df_clean['MINMAX_NUM_CHLDRN'][(df_clean['MINMAX_NUM_CHLDRN'].notnull())&(df_clean['LIFE_STG_New married couple'] == 1)])
nmc_notnull_valuecount = df_clean['MINMAX_NUM_CHLDRN'][(df_clean['MINMAX_NUM_CHLDRN'].notnull())&(df_clean['LIFE_STG_New married couple'] == 1)].value_counts(dropna = False)
print('New married couple not null:',nmc_notnull,'row', '\n With values count:', nmc_notnull_valuecount,'row')

In [ ]:
df_clean['STD_NUM_CHLDRN'] = np.where((df_clean['STD_NUM_CHLDRN'].isnull()) & 
                                         (df_clean['LIFE_STG_New married couple'] == 1),
                                         -1.189127, df_clean['STD_NUM_CHLDRN'])

In [ ]:
df_clean['MINMAX_NUM_CHLDRN'] = np.where((df_clean['MINMAX_NUM_CHLDRN'].isnull()) & 
                                         (df_clean['LIFE_STG_New married couple'] == 1),
                                         0, df_clean['MINMAX_NUM_CHLDRN'])

2. Observation with 'LIFE_STG' column : Family with children <br>
We see that histrogram skewed right so we decided to fill NaN as 0.014493 (median & mode) or 1 child for Normalization<br>
and -0.483409 for Standardization<br>

In [ ]:
df_clean['MINMAX_NUM_CHLDRN'][(df_clean['MINMAX_NUM_CHLDRN'].notnull()) & 
                              (df_clean['LIFE_STG_Family with children'] == 1)].describe()

In [ ]:
df_clean['STD_NUM_CHLDRN'] = np.where((df_clean['STD_NUM_CHLDRN'].isnull()) & 
                                         (df_clean['LIFE_STG_Family with children'] == 1), 
                                         -0.483409, df_clean['STD_NUM_CHLDRN'])

In [ ]:
df_clean['MINMAX_NUM_CHLDRN'] = np.where((df_clean['MINMAX_NUM_CHLDRN'].isnull()) & 
                                         (df_clean['LIFE_STG_Family with children'] == 1), 
                                         0.014493, df_clean['MINMAX_NUM_CHLDRN'])

3. Observation with 'LIFE_STG' column : Retirement / Family with adolescents<br>
We see that histrogram skewed right so we decided to fill NaN as 0.028986 (median & mode) or 2 children for Normalization<br>
and 0.222309 for Standardization<br>

In [ ]:
df_clean['STD_NUM_CHLDRN'] = np.where((df_clean['STD_NUM_CHLDRN'].isnull()) & (
    (df_clean['LIFE_STG_Retirement'] == 1) | (df_clean['LIFE_STG_Family with adolescents'] == 1)), 
                                         0.222309, df_clean['STD_NUM_CHLDRN'])

In [ ]:
df_clean['MINMAX_NUM_CHLDRN'] = np.where((df_clean['MINMAX_NUM_CHLDRN'].isnull()) & (
    (df_clean['LIFE_STG_Retirement'] == 1) | (df_clean['LIFE_STG_Family with adolescents'] == 1)), 
                                         0.028986, df_clean['MINMAX_NUM_CHLDRN'])

We dropped out one column per categorical data for preventing redundancy from get dummies.

In [ ]:
df_clean.drop(columns = 'LIFE_STG_Family with adolescents', axis = 1, inplace = True)

---

<h3> 2. Dropping missing records </h3>

---

<b> <i> DUR_TO_NEXT_ME_DT </i> </b>

We found that 'STD_DUR_TO_NEXT_ME_DT' and 'MINMAX_DUR_TO_NEXT_ME_DT' had low value : NaN compared to all distribution <br> <br>
We will check that 'STD_DUR_TO_NEXT_ME_DT' and 'MINMAX_DUR_TO_NEXT_ME_DT' value = NaN has affect on all target = 1 

From result, it had no effect -> We decided to drop 'STD_DUR_TO_NEXT_ME_DT' and 'MINMAX_DUR_TO_NEXT_ME_DT' value = NaN

In [ ]:
df_clean.dropna(subset = ['STD_DUR_TO_NEXT_ME_DT'], inplace = True)
df_clean.dropna(subset = ['MINMAX_DUR_TO_NEXT_ME_DT'], inplace = True)

---

<b> <i> PPREM_LTD </i> </b>

We found that 'PPREM_LTD' had low value : NaN compared to all distribution

We will check that 'STD_PPREM_LTD' and 'MINMAX_PPREM_LTD' value = NaN has affect on all target = 1 

From result, it had no effect -> We decided to drop 'STD_PPREM_LTD' and 'MINMAX_PPREM_LTD' value = NaN

In [ ]:
df_clean.dropna(subset = ['STD_PPREM_LTD'], inplace = True)
df_clean.dropna(subset = ['MINMAX_PPREM_LTD'], inplace = True)

---

<h3> 3. Dropping variables </h3>

---

<b> <i> NUM_FULLY_PC_INF </i> </b>

We found that 'STD_NUM_FULLY_PC_INF' and 'MINMAX_NUM_FULLY_PC_INF' had only value = 0 <br> <br>
We decided to drop 'STD_NUM_FULLY_PC_INF' and 'MINMAX_NUM_FULLY_PC_INF' column because the column which had all same value were impractical for training model

In [ ]:
df_clean.drop(columns = ['STD_NUM_FULLY_PC_INF'], axis = 1, inplace = True)
df_clean.drop(columns = ['MINMAX_NUM_FULLY_PC_INF'], axis = 1, inplace = True)

---

<b> <i> NUM_PO_OTH_PROD_INF </i> </b>

We found that 'STD_NUM_PO_OTH_PROD_INF' and 'MINMAX_NUM_PO_OTH_PROD_INF' had only value = 0 <br> <br>
We decided to drop 'STD_NUM_PO_OTH_PROD_INF' and 'MINMAX_NUM_PO_OTH_PROD_INF' column because the column which had all same value were impractical for training model

In [ ]:
df_clean.drop(columns = ['STD_NUM_PO_OTH_PROD_INF'], axis = 1, inplace = True)
df_clean.drop(columns = ['MINMAX_NUM_PO_OTH_PROD_INF'], axis = 1, inplace = True)

---

<b> <i> PCT_PAR_PO_INF </i> </b>

We found that 'STD_PCT_PAR_PO_INF' and 'MINMAX_PCT_PAR_PO_INF' had only value = 0 <br> <br>
We decided to drop 'STD_PCT_PAR_PO_INF' and 'MINMAX_PCT_PAR_PO_INF' column because the column which had all same value were impractical for training model

In [ ]:
df_clean.drop(columns = ['STD_PCT_PAR_PO_INF'], axis = 1, inplace = True)
df_clean.drop(columns = ['MINMAX_PCT_PAR_PO_INF'], axis = 1, inplace = True)

---

<b> <i> SA_HB_BNFT_INF </i> </b>

We found that 'STD_SA_HB_BNFT_INF' and 'MINMAX_SA_HB_BNFT_INF' had only value = 0 <br> <br>
We decided to drop 'STD_SA_HB_BNFT_INF' and 'MINMAX_SA_HB_BNFT_INF' column because the column which had all same value were impractical for training model       

In [ ]:
df_clean.drop(columns = ['STD_SA_HB_BNFT_INF'], axis = 1, inplace = True)
df_clean.drop(columns = ['MINMAX_SA_HB_BNFT_INF'], axis = 1, inplace = True)

---

<b> <i> SA_OTH_BNFT_INF </i> </b> 

We found that 'STD_SA_OTH_BNFT_INF' and 'MINMAX_SA_OTH_BNFT_INF' had only value = 0 <br> <br>
We decided to drop 'STD_SA_OTH_BNFT_INF' and 'MINMAX_SA_OTH_BNFT_INF' column because the column which had all same value were impractical for training model

In [ ]:
df_clean.drop(columns = ['STD_SA_OTH_BNFT_INF'], axis = 1, inplace = True)
df_clean.drop(columns = ['MINMAX_SA_OTH_BNFT_INF'], axis = 1, inplace = True)

---

<h3> 4. Dropping variables with condition </h3>

---

Our criteria : We decided to drop column which has these conditions : <br>
1. has 1 possible outcome more than 99 % of data (from cell above) <br>
2. and that possible outcome has relationship with all target = 1 more than 99 % (from cell below)

We found that the possible outcome of 'MINMAX_AVG_PPREM_ST_AMT' has target class more than 99% of all target class <br>
We decided to drop 'MINMAX_AVG_PPREM_ST_AMT' column <br>
Similarly, we dropped 'STD_AVG_PPREM_ST_AMT' column

In [ ]:
df_clean.drop(columns = ['MINMAX_AVG_PPREM_ST_AMT'], axis = 1, inplace = True)
df_clean.drop(columns = ['STD_AVG_PPREM_ST_AMT'], axis = 1, inplace = True)

We found that the possible outcome of 'MINMAX_PPREM_OTH_BNFT_L1Y' has target class more than 99% of all target class <br>
We decided to drop 'MINMAX_PPREM_OTH_BNFT_L1Y' column <br>
Similarly, we dropped 'STD_PPREM_OTH_BNFT_L1Y' column

In [ ]:
df_clean.drop(columns = ['MINMAX_PPREM_OTH_BNFT_L1Y'], axis = 1, inplace = True)
df_clean.drop(columns = ['STD_PPREM_OTH_BNFT_L1Y'], axis = 1, inplace = True)

Moreover, we decided to drop 'MINMAX_NUM_REJ_CA_P3Y' and 'STD_NUM_REJ_CA_P3Y' because the majority of value = NaN

In [ ]:
df_clean.drop(columns = ['MINMAX_NUM_REJ_CA_P3Y'], axis = 1, inplace = True)
df_clean.drop(columns = ['STD_NUM_REJ_CA_P3Y'], axis = 1, inplace = True)

---

<a class='anchor' id='Step3HHCV'> <h3> <b> Step 3 : Handle High Correlation Variables</b> </h3> </a>

---

STD<br>
Remove Column form One-Hot_encoding because it will give 1.00 correlation in each category itself

In [ ]:
df_std = df_clean.copy()

In [ ]:
#df.columns.get_loc("MINMAX_CUST_TIER")
df_std.drop(columns = list(df_std.columns[104:241]), axis = 1, inplace = True) #MINMAX_AGE:MINMAX_CUST_TIER

In [ ]:
df_std.drop(columns = list(df_std.columns[0:6]), axis = 1, inplace = True)

In [ ]:
def correlation(df, threshold):
    col_corr = set()
    corr_matrix = df.corr().abs()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
                if colname in df.columns:
                    del df[colname]
    return df
    
correlation(df_std, 0.8) #0.8 is very strong correlation

---

<a class='anchor' id='Step4OH'> <h3> <b> Step 4 : Outlier Handling</b> </h3> </a>

---

In [ ]:
df_std['PH_ID_BASE'] = df_clean['PH_ID_BASE']

In [ ]:
df_std_oh = df_std.copy()

In [ ]:
i_count = 0
df_std_oh = df_std
for i in df_std.columns:
    #print(i)
    threshold = df_std_oh[i].std()*10.5
    
    df_std_oh = df_std_oh[(df_std_oh[df_std_oh.columns[int(i_count)]] < threshold)]
    df_std_oh = df_std_oh.reset_index(drop=True)
    #print(df_std_oh.shape)
    i_count += 1

Merge dataset

In [ ]:
dfc = df_clean.copy()
dfc.drop(columns = list(dfc.columns[6:202]), axis = 1, inplace = True)
dfc.drop(columns = list(dfc.columns[44:46]), axis = 1, inplace = True)

In [ ]:
df_clean = pd.merge(dfc, df_std_oh, on='PH_ID_BASE')

---

<a class='anchor' id='3.ModelDevelopment'> <h2> ---------- < 3. Model Developing > ---------- </h2> </a>

<a class='anchor' id='Step1VS'><h3> <b> Step 1 : Variable Selection</b> </h3> </a>

<b> Variable Selection With RF </b>

In [ ]:
df_clean.drop(columns = ['PH_ID_BASE'])
x = df_clean.drop(columns = list(df_clean.columns[7:16])) #HLH_TRG_FLG:UNUL_KB_TRG_FLG
y = df_clean['WL_AG_TRG_FLG']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

In [ ]:
arr = rf.feature_importances_
sorted_index_array = np.argsort(arr)
sorted_array = arr[sorted_index_array]

In [ ]:
rslt = sorted_array[-40:]
rslt_index = sorted_index_array[-40:]

In [ ]:
col_select = x.columns.to_numpy()[np.array(rslt_index)]

In [ ]:
df_model = pd.DataFrame(df_clean, columns=col_select)

In [ ]:
df_model['PH_ID_BASE'] = df_clean['PH_ID_BASE']
df_model['WL_AG_TRG_FLG'] = df_clean['WL_AG_TRG_FLG']

In [ ]:
#Remove feature that have some correlation
df_model.drop(columns = ['STD_NUM_PO_PURCH_L2Y', 'STD_NUM_RIDER_PURCH_L3Y', 'STD_NUM_RIDER_PURCH_L2Y'], axis = 1, inplace = True)

---

<a class='anchor' id='Step234'><h3> <b> Step 2,3,4 : Model Selection - Model Validation - Performance Testing</b> </h3> </a>

In [ ]:
x = df_model.drop(columns= list(df_model.columns[37:42]), axis = 1)
y = df_model['WL_AG_TRG_FLG']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
x_train_valid, x_valid, y_train_valid, y_valid = train_test_split(x_train, y_train, test_size = 0.125, random_state = 42)
#0.7 0.1 0.2

<b>Random Forest Classifier</b>

In [ ]:
def create_rf_grid(x_train, y_train, x_test, y_test):

    #Create Model
    rf = RandomForestClassifier(n_estimators=100 , max_depth=10 , criterion='gini', min_samples_split=10, 
                                min_samples_leaf=4, max_features='auto', class_weight='balanced', random_state=42, n_jobs=-1)
    rf.fit(x_train, y_train)
    y_pred = rf.predict(x_train)
    #Matrics
    acc = accuracy_score(y_train, y_pred)
    bl_acc = balanced_accuracy_score(y_train, y_pred)
    recall = recall_score(y_train, y_pred)
    prec = precision_score(y_train, y_pred)
    f3 = fbeta_score(y_train, y_pred, beta=3.0)
    print('Training_set\n')
    print(classification_report(y_train, y_pred))
    print(confusion_matrix(y_train, y_pred))
    print('Accuracy:', acc)
    print('Balanced Accuracy:', bl_acc)
    print('Recall:', recall)
    print('Precision:', prec)
    print('F3:', f3)
    print('-'*50)
    
    y_pred_test = rf.predict(x_test)
    acc_test = accuracy_score(y_test, y_pred_test)
    bl_acc_test = balanced_accuracy_score(y_test, y_pred_test)
    recall_test = recall_score(y_test, y_pred_test)
    prec_test = precision_score(y_test, y_pred_test)
    f3_test = fbeta_score(y_test, y_pred_test, beta=3.0)
    print('\nTest_set\n')
    print(classification_report(y_test, y_pred_test))
    print(confusion_matrix(y_test, y_pred_test))
    print('Accuracy:', acc_test)
    print('Balanced Accuracy:', bl_acc_test)
    print('Recall:', recall_test)
    print('Precision:', prec_test)
    print('F3:', f3_test)
    print('-'*50)
    
    feat_importances = pd.Series(rf.feature_importances_, index=x.columns)
    print(feat_importances.sort_values(ascending=False).head(10))

---

<b>Test Set</b>

In [ ]:
create_rf_grid(x_train, y_train, x_test, y_test)

---

<a class='anchor' id='4.ModelImplementation'><h2> ---------- < 4. Model Implementation > ---------- </h2> </a>

<a class='anchor' id='ICM'> <h3> <b> Interpret with Confusion Matrix</b> </h2> </a>

In [ ]:
rf = RandomForestClassifier(n_estimators=100 , max_depth=10 , criterion='gini', min_samples_split=10, 
                            min_samples_leaf=4 , class_weight='balanced', random_state=42, n_jobs=-1)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
y_list = y_test.to_list()
columns_list = list(x_train.columns)

In [ ]:
#TP predict 1, actual 1
df_tp = pd.DataFrame(columns=columns_list)

for i in range(len(y_pred)):
    dict = {}
    if (y_pred[i] == 1) & (y_list[i] == 1):
        for j in x_test.iloc[[i]]:
            dict[j] = float(x_test.iloc[[i]][j])
        df_tp = df_tp.append(dict, ignore_index = True)

In [ ]:
df_tp['Target']='TP'

In [ ]:
#TN predict 0, Actual 0
df_tn = pd.DataFrame(columns=columns_list)

for i in range(len(y_pred)):
    dict = {}
    if (y_pred[i] == 0) & (y_list[i] == 0):
        for j in x_test.iloc[[i]]:
            dict[j] = float(x_test.iloc[[i]][j])
        df_tn = df_tn.append(dict, ignore_index = True)

In [ ]:
df_tn['Target']='TN'

In [ ]:
#FP predict 1, Actual 0
df_fp = pd.DataFrame(columns=columns_list)

for i in range(len(y_pred)):
    dict = {}
    if (y_pred[i] == 1) & (y_list[i] == 0):
        for j in x_test.iloc[[i]]:
            dict[j] = float(x_test.iloc[[i]][j])
        df_fp = df_fp.append(dict, ignore_index = True)

In [ ]:
df_fp['Target'] = 'FP'

In [ ]:
#FN predict 0, Actual 1
df_fn = pd.DataFrame(columns=columns_list)

for i in range(len(y_pred)):
    dict = {}
    if (y_pred[i] == 0) & (y_list[i] == 1):
        for j in x_test.iloc[[i]]:
            dict[j] = float(x_test.iloc[[i]][j])
        df_fn = df_fn.append(dict, ignore_index = True)

In [ ]:
df_fn['Target'] = 'FN'

In [ ]:
df_confuse = pd.concat([df_tn, df_tp, df_fn, df_fp], ignore_index=True)

In [ ]:
col_list = list(df_confuse.columns)
col_list.remove('Target')

In [ ]:
for i in col_list:
    sns.boxplot(x='Target', y=i, data=df_confuse)
    plt.show()

---

<a class='anchor' id='ISV'> <h3> <b> Interpret with SHAP Value</b> </h3> </a>

In [ ]:
shap_values = shap.TreeExplainer(rf).shap_values(x_test)

In [ ]:
shap.summary_plot(shap_values[1], x_test, plot_type="violin")

In [ ]:
x_test_copy = x_test.copy()
x_test_copy['predicted'] = rf.predict(x_test)
shap_values_x_test_copy = shap.TreeExplainer(rf).shap_values(x_test_copy)

In [ ]:
listshap_40 = x_test_copy.columns
listpred = ['predicted']

In [ ]:
for i in listshap_40:
    for j in listpred:
        print(i, j)
        shap.dependence_plot(i, shap_values_x_test_copy[1], x_test_copy, interaction_index=j, alpha=0.3, cmap = plt.get_cmap("cool"))
        print('-'*100)
# dim 2 (i) dim 3 (j)

---